In [ ]:
import os
import shutil
import numpy as np # linear algebra
import matplotlib.pyplot as plt # to show images
from PIL import Image # to read images
import matplotlib.patches as patches
import os
import cv2
import random
import csv
import pandas as pd

In [ ]:
# Create the txts in another directory
# Read the CSV file into a Pandas DataFrame
csv_PATH = 'C:\\Users\\marru\\Documents\\Magistrale\\Secondo_Semestre\\I_Systems\\Project\\annotations.csv'
src_img = 'C:\\Users\\marru\\Pictures\\images_backup'
output_dir = 'C:\\Users\\marru\\Documents\\Magistrale\\Secondo_Semestre\\I_Systems\\Project\\txts_chinese_dataset'
if not os.path.exists(output_dir):
	os.mkdir(output_dir)

df = pd.read_csv(csv_PATH)
filenames_col = df['file_name']
filenames= filenames_col.to_list()
print(filenames_col)

for index, img in enumerate(filenames):
    #print(img)
    #print(index)
    name = img[:-4]
    print(name)

    image_path = os.path.join(src_img, img)

    # reading images
    Image1 = cv2.imread(image_path)
    size = Image1.shape
    print("size:", size)

    img_width = size[1]
    img_height = size[0]

    selected_row = df.iloc[index]

    x_min = selected_row['x1']
    y_min = selected_row['y1']
    x_max = selected_row['x2']
    y_max = selected_row['y2']
    w = x_max - x_min
    h = y_max - y_min
    img_class = selected_row['category']

    x_center = (x_min + x_max)/ 2
    y_center = (y_min + y_max)/ 2

    # conversion in yolo format
    new_x_center_yolo = x_center / img_width
    new_y_center_yolo = y_center / img_height
    new_w_yolo = w / img_width
    new_h_yolo = h / img_height

    # crea nuovo txt con il bounding box del crop
    nameFile_txt = name + ".txt"

    save_path_txt = os.path.join(output_dir, nameFile_txt)
    new_txt = open(save_path_txt, "w")

    string_value = str(new_x_center_yolo)  + ' ' + str(new_y_center_yolo) + ' ' + str(new_w_yolo) + ' ' + str(new_h_yolo)
    print(string_value)
    new_txt.write(string_value)
    new_txt.close()

In [ ]:
# Merge txts and images in same dir
image_PATH = 'C:\\Users\\marru\\Documents\\Magistrale\\Secondo_Semestre\\I_Systems\\Project\\chinese_dataset'
txt_PATH = 'C:\\Users\\marru\\Documents\\Magistrale\\Secondo_Semestre\\I_Systems\\Project\\txts_chinese_dataset'
new_dataset_PATH = 'C:\\Users\\marru\\Documents\\Magistrale\\Secondo_Semestre\\I_Systems\\Project\\final_datasets\\pre_elaborated_data_chinese'
if not os.path.exists(new_dataset_PATH):
	os.mkdir(new_dataset_PATH)

images = os.listdir(image_PATH)
txts = os.listdir(txt_PATH)
for image in images:
    shutil.copy(os.path.join(image_PATH, image), new_dataset_PATH)
for txt in txts:
    shutil.copy(os.path.join(txt_PATH, txt), new_dataset_PATH)

In [ ]:
#---------------------------IF NEEDED----------------------------------------
# DIVIDE THE DATASET AND CREATE THE NEW DIRS BEFORE SELECTING THE RIGHT CLASSES
import os
import shutil
import random

#Divide all images based on the class and create a dir for each
PATH = 'C:\\Users\\marru\\Documents\\Magistrale\\Secondo_Semestre\\I_Systems\\Project\\final_datasets\\pre_elaborated_data_chinese'
all_images = os.listdir(PATH)
dir_PATH = 'C:\\Users\\marru\\Documents\\Magistrale\\Secondo_Semestre\\I_Systems\\Project\\final_datasets\\division_data_chinese'
if not os.path.exists(dir_PATH):
	os.mkdir(dir_PATH)
previous_image = "null"
for filename in all_images:
    short_name = filename[:3]
    if short_name != previous_image:
        new_PATH = os.path.join(dir_PATH, short_name)
        os.mkdir(new_PATH)
    shutil.copy(os.path.join(PATH, filename), new_PATH)
    previous_image = short_name

# Count all images for each class
PATH = 'C:\\Users\\marru\\Documents\\Magistrale\\Secondo_Semestre\\I_Systems\\Project\\final_datasets\\division_data_chinese'
all_classes = os.listdir(PATH)
for dirname in all_classes:
    num_images = len(os.listdir(os.path.join(PATH, dirname)))
    print('Images in class', dirname, ':', num_images)

In [ ]:
#ADD CODE TO EACH IMAGE
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import random
import os

#Images
image_dir = 'C:\\Users\\marru\\Documents\\Magistrale\\Secondo_Semestre\\I_Systems\\Project\\final_datasets\\pre_elaborated_data_chinese'
MANDATORY = ['020','021','022','023','024','026','027','030'] #Code: 0
PROHIBITION = ['054','055'] #Code: 1
SPEED_LIMIT = ['002','003','004','005','006','007'] #Code: 2
WARNING = ['032','034','035','036','037','038','039','040','041','042','043','044','045','046','047','048','049','050','051'] #Code: 3

new_dir = 'C:\\Users\\marru\\Documents\\Magistrale\\Secondo_Semestre\\I_Systems\\Project\\final_datasets\\elaborated_data_chinese'
if not os.path.exists(new_dir):
	os.mkdir(new_dir)

# List all image filenames (without extensions)
image_filenames = [filename[:-4] for filename in os.listdir(image_dir) if filename.endswith('.png')]

for image in image_filenames:
    code = -1;
    prov_name = image[0:3]
    if prov_name in MANDATORY:
    	code = 0
    elif prov_name in PROHIBITION:
    	code = 1
    elif prov_name in SPEED_LIMIT:
    	code = 2
    elif prov_name in WARNING:
    	code = 3
    else:
    	continue
    old_image_path = os.path.join(image_dir, image + '.png')
    old_information_path = os.path.join(image_dir, image + '.txt')
	#PROCESSING REGRESSION BOX
    # Load a new image, save new files and preprocess it
    new_image = Image.open(old_image_path)
    new_image_path = os.path.join(new_dir, image + '.jpg')
    information_path = os.path.join(new_dir, image + '.txt')
    new_image.save(new_image_path)
    file_to_read = open(old_information_path, 'r')
    line_in_file = file_to_read.readline()
    box = [float(string) for string in line_in_file.strip().split(' ')]

    # Extract box coordinates
    x, y, width, height = box
    #Save as string to write in new txt
    final_line = "%d %f %f %f %f" % (code, x, y, width, height)

    with open(information_path, 'w') as f:
        f.write(final_line)

In [1]:
# Used to divide in the 4 classes the original elaborated images
import os
import shutil

chinese_PATH = 'C:\\Users\\marru\\Documents\\Magistrale\\Secondo_Semestre\\I_Systems\\Project\\final_datasets\\elaborated_data_chinese'
ita_PATH = 'C:\\Users\\marru\\Documents\\Magistrale\\Secondo_Semestre\\I_Systems\\Project\\final_datasets\\elaborated_data_ita'
all_PATH= [chinese_PATH, ita_PATH]

mandatory_PATH = 'C:\\Users\\marru\\Documents\\Magistrale\\Secondo_Semestre\\I_Systems\\Project\\final_datasets\\mandatory_signs'
prohibition_PATH = 'C:\\Users\\marru\\Documents\\Magistrale\\Secondo_Semestre\\I_Systems\\Project\\final_datasets\\prohibition_signs'
speedlimit_PATH = 'C:\\Users\\marru\\Documents\\Magistrale\\Secondo_Semestre\\I_Systems\\Project\\final_datasets\\speedlimit_signs'
warning_PATH = 'C:\\Users\\marru\\Documents\\Magistrale\\Secondo_Semestre\\I_Systems\\Project\\final_datasets\\warning_signs'
category_PATHS = [mandatory_PATH, warning_PATH, speedlimit_PATH, prohibition_PATH]
for path in category_PATHS:
    if not os.path.exists(path):
	    os.mkdir(path)

for image_PATH in all_PATH:
    save_PATH=''
    image_names = [filename[:-4] for filename in os.listdir(image_PATH) if filename.endswith('.jpg')]
    for image_filename in image_names:
        current_image_path = os.path.join(image_PATH, image_filename + '.jpg')
        current_information_path = os.path.join(image_PATH, image_filename + '.txt')
        print(image_filename)

        # Get box information in YOLO format
        file_to_read = open(current_information_path, 'r')
        line_in_file = file_to_read.readline()
        box = [float(string) for string in line_in_file.strip().split(' ')]
        save_code = int(box[0])
        if save_code==0:
                save_PATH=mandatory_PATH
        elif save_code==1:
                save_PATH=prohibition_PATH
        elif save_code==2:
                save_PATH=speedlimit_PATH
        elif save_code==3:
                save_PATH=warning_PATH

        save_image_PATH = os.path.join(save_PATH, image_filename + '.jpg')
        save_information_PATH = os.path.join(save_PATH, image_filename + '.txt')
        shutil.copy(current_image_path,save_image_PATH)
        shutil.copy(current_information_path, save_information_PATH)

002_0001
002_0001_j
002_0002
002_0002_j
002_0003
002_0003_j
002_0004
002_0004_j
002_0005
002_0005_j
002_0006
002_0006_j
002_0007
002_0007_j
002_0008
002_0008_j
002_0009
002_0009_j
002_0010
002_0010_j
002_0011
002_0011_j
002_0012
002_0012_j
002_0013
002_0013_j
002_0014
002_0015
002_0016
002_0017
002_0018
002_0019
002_0020
002_0021
002_0022
002_0023
002_0024
002_0025
002_0026
002_0027
002_0028
002_0029
002_0030
002_0031
002_0032
002_0033
002_0034
002_0035
002_0036
002_0037
002_0038
002_0039
002_1_0001
002_1_0001_1_j
002_1_0002
002_1_0002_1_j
002_1_0003
002_1_0003_1_j
002_1_0004
002_1_0004_1_j
002_1_0005
002_1_0005_1_j
002_1_0006
002_1_0006_1_j
002_1_0007
002_1_0007_1_j
002_1_0008
002_1_0008_1_j
002_1_0009
002_1_0009_1_j
002_1_0010
002_1_0010_1_j
002_1_0011
002_1_0011_1_j
002_1_0012
002_1_0012_1_j
002_1_0013
002_1_0013_1_j
002_1_0014
002_1_0015
002_1_0016
002_1_0017
002_1_0018
002_1_0019
002_1_0020
002_1_0021
002_1_0022
002_1_0023
002_1_0024
002_1_0025
002_1_0026
002_1_0027
002_1_0028
002

In [2]:
# Divide all the images and txt in 80 (training), 10 (validation), 10 (test)
import os
import shutil
import random

mandatory_PATH = 'C:\\Users\\marru\\Documents\\Magistrale\\Secondo_Semestre\\I_Systems\\Project\\final_datasets\\mandatory_signs'
prohibition_PATH = 'C:\\Users\\marru\\Documents\\Magistrale\\Secondo_Semestre\\I_Systems\\Project\\final_datasets\\prohibition_signs'
speedlimit_PATH = 'C:\\Users\\marru\\Documents\\Magistrale\\Secondo_Semestre\\I_Systems\\Project\\final_datasets\\speedlimit_signs'
warning_PATH = 'C:\\Users\\marru\\Documents\\Magistrale\\Secondo_Semestre\\I_Systems\\Project\\final_datasets\\warning_signs'
augmented_mandatory_PATH = 'C:\\Users\\marru\\Documents\\Magistrale\\Secondo_Semestre\\I_Systems\\Project\\final_datasets\\augmented_mandatory_signs'
augmented_prohibition_PATH = 'C:\\Users\\marru\\Documents\\Magistrale\\Secondo_Semestre\\I_Systems\\Project\\final_datasets\\augmented_prohibition_signs'
augmented_speedlimit_PATH = 'C:\\Users\\marru\\Documents\\Magistrale\\Secondo_Semestre\\I_Systems\\Project\\final_datasets\\augmented_speedlimit_signs'
augmented_warning_PATH = 'C:\\Users\\marru\\Documents\\Magistrale\\Secondo_Semestre\\I_Systems\\Project\\final_datasets\\augmented_warning_signs'
all_augmented_PATHS = [augmented_mandatory_PATH, augmented_warning_PATH, augmented_speedlimit_PATH, augmented_prohibition_PATH]
category_PATHS = [mandatory_PATH, warning_PATH, speedlimit_PATH, prohibition_PATH]

train_PATH = 'C:\\Users\\marru\\Documents\\Magistrale\\Secondo_Semestre\\I_Systems\\Project\\final_datasets\\train'
validation_PATH = 'C:\\Users\\marru\\Documents\\Magistrale\\Secondo_Semestre\\I_Systems\\Project\\final_datasets\\validation'
test_PATH = 'C:\\Users\\marru\\Documents\\Magistrale\\Secondo_Semestre\\I_Systems\\Project\\final_datasets\\test'
training_PATHS=[train_PATH, validation_PATH, test_PATH]
for path in training_PATHS:
    if not os.path.exists(path):
	    os.mkdir(path)

for image_PATH in category_PATHS:
    counter = 0
    image_names = [filename[:-4] for filename in os.listdir(image_PATH) if filename.endswith('.jpg')]
    random.shuffle(image_names)
    for image_filename in image_names:
        counter+=1
        current_image_path = os.path.join(image_PATH, image_filename + '.jpg')
        current_information_path = os.path.join(image_PATH, image_filename + '.txt')
        print(image_filename)
        if counter <= 200:
            shutil.copy(current_image_path, validation_PATH)
            shutil.copy(current_information_path, validation_PATH)
        elif 400 >= counter > 200:
            shutil.copy(current_image_path, test_PATH)
            shutil.copy(current_information_path, test_PATH)
        else:
            shutil.copy(current_image_path, train_PATH)
            shutil.copy(current_information_path, train_PATH)

for image_PATH in all_augmented_PATHS:
    image_names = [filename[:-4] for filename in os.listdir(image_PATH) if filename.endswith('.jpg')]
    for image_filename in image_names:
        current_image_path = os.path.join(image_PATH, image_filename + '.jpg')
        current_information_path = os.path.join(image_PATH, image_filename + '.txt')
        print(image_filename)
        shutil.copy(current_image_path, train_PATH)
        shutil.copy(current_information_path, train_PATH)

00371_0
01358_0
026_0062
027_1_0013
030_0065
026_0009_j
00078_0
00102_0
00936_0
00590_3
030_1_0003_1_j
01237_0
030_0010
024_0022
026_0004_j
024_1_0015
026_1_0042_1_j
01524_0
024_0016
030_0022
00954_0
027_0010_j
01401_0
024_1_0011
022_0003_j
01360_0
026_1_0048_1_j
021_1_0003_1_j
00594_0
00394_0
00489_0
00127_0
022_1_0001_1_j
00351_1
024_1_0016
00502_0
01270_0
026_0005_j
026_0008_j
030_0049
00678_1
027_0008
00437_0
024_0031
030_0028
030_0005_j
01343_0
00165_0
00554_1
026_1_0040
00910_0
01486_2
026_1_0047
030_1_0005_1_j
026_0014
026_0033_j
030_0041
00906_0
00570_2
021_1_0001_1_j
02125_0
00589_0
00292_1
026_1_0060
024_0001
030_0008
024_1_0031
030_1_0055
024_1_0001
024_1_0038
01478_0
00948_0
00121_0
026_0015
024_1_0029
026_0030_j
00176_0
00361_1
030_0008_j
030_0069
01397_0
027_0010
026_1_0046
026_0037_j
026_1_0018
024_0033
02028_0
030_0036
026_1_0008
00962_0
026_1_0055
02014_0
026_0060_j
030_0054
026_1_0044
030_1_0010
00888_1
026_1_0033_1_j
026_1_0031_1_j
030_1_0006_1_j
02074_0
026_0014_j
0